#**. Executar Extração**

##**. Instalação**

In [ ]:
# @title Instalação
!pip install pandas
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**. Setup**

In [ ]:
# @title Setup

from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
from google.cloud import storage
import pandas as pd
import os

project = 'athenas-364914' 
location = 'US' 
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

##**. Visualização de Querry**

In [ ]:
# @title SQL de consulta

job = client.get_job('bquxjob_6f8a23e9_183c3bd4c53') 
print(job.query)

SELECT
  `athenas-364914.Athenas2.Ligacoes_dist`.midia AS TIPO_MIDIA,
  `athenas-364914.Athenas2.Ligacoes_dist`.description AS STATUS_FINAL,
  FORMAT_DATETIME('%D',`athenas-364914.Athenas2.Ligacoes_dist`.created_at)AS DATA,
  COUNT(`athenas-364914.Athenas2.Ligacoes_dist`.status_id) AS QUANTIDADE
FROM
  `athenas-364914.Athenas2.Ligacoes_dist`
WHERE
  `athenas-364914.Athenas2.Ligacoes_dist`.status_id IN
  (57,100)
GROUP BY
  DATA,
  TIPO_MIDIA,
  STATUS_FINAL;


##**. Visualização de Dados**

In [ ]:
# @title Visualização

job = client.get_job('bquxjob_6f8a23e9_183c3bd4c53') 
results = job.to_dataframe()
results

,TIPO_MIDIA,STATUS_FINAL,DATA,QUANTIDADE
0,SMS,Atendimento,None,2
1,ORGANIC,Atendimento,08/06/20,6
2,ORGANIC,Atendimento,None,23
3,ORGANIC,Atendimento,04/06/20,3
4,ORGANIC,Atendimento,01/06/20,6
5,ORGANIC,Atendimento,02/06/20,4
6,ORGANIC,Atendimento,06/06/20,1
7,ORGANIC,Atendimento,05/06/20,2
8,ORGANIC,Atendimento,11/06/20,2
9,ORGANIC,Atendimento,03/06/20,2


extração

##**. Extração dos Dados**

In [ ]:
# @title Extração

df = results
df.to_excel('/content/ligacoes_recep.xlsx', index = False)

##**. Conexão ao GCP**

In [ ]:
# @title Conexão ao GCP

serviceAccount = '/content/athenas-364914-4e16df1244dc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

##**. Inserindo Excell no GCP**

In [ ]:
# @title Inserindo Excell no GCP


client = storage.Client.from_service_account_json(json_credentials_path='/content/athenas-364914-4e16df1244dc.json')
bucket = client.get_bucket('saida_modelagem')
object_name_in_gcs_bucket = bucket.blob('ligacoes_recep.xlsx')
object_name_in_gcs_bucket.upload_from_filename('ligacoes_recep.xlsx')
